In [1]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend.tensorflow_backend as K

X_train, X_test, y_train, y_test = np.load('../numpy_data/multi_image_data.npy', allow_pickle=True)
print(X_train.shape)
print(X_train[0].shape)

Using TensorFlow backend.
(145800, 32, 32, 3)
(32, 32, 3)


In [40]:
with K.tf_ops.device('/device:GPU:0'):
    #모델생성
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3,3), padding="same", activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    model_dir = '../model'

    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    model_path = model_dir + '/model2.h5'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=2, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [8]:
X_train.shape[1:]

(32, 32, 3)

In [41]:
with K.tf_ops.device('/device:GPU:0'):
    history = model.fit(X_train, y_train, batch_size=512, epochs=50, validation_data=(X_test, y_test),validation_split=0.2, callbacks=[checkpoint, early_stopping])

Train on 238728 samples, validate on 79576 samples
Epoch 1/50
238728/238728 [==============================] - 418s 2ms/step - loss: 4.3246 - accuracy: 0.0591 - val_loss: 3.6942 - val_accuracy: 0.1545

Epoch 00001: val_loss improved from inf to 3.69424, saving model to ../model/model2.h5
Epoch 2/50
238728/238728 [==============================] - 416s 2ms/step - loss: 3.5201 - accuracy: 0.1653 - val_loss: 2.8595 - val_accuracy: 0.3095

Epoch 00002: val_loss improved from 3.69424 to 2.85954, saving model to ../model/model2.h5
Epoch 3/50
238728/238728 [==============================] - 415s 2ms/step - loss: 3.1166 - accuracy: 0.2349 - val_loss: 2.5112 - val_accuracy: 0.3829

Epoch 00003: val_loss improved from 2.85954 to 2.51119, saving model to ../model/model2.h5
Epoch 4/50
238728/238728 [==============================] - 412s 2ms/step - loss: 2.8704 - accuracy: 0.2811 - val_loss: 2.2568 - val_accuracy: 0.4593

Epoch 00004: val_loss improved from 2.51119 to 2.25682, saving model to ../m

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = '../model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/model2.h5'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=2, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [42]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, "../model/")

In [11]:
!pip install tensorflowjs

  Using cached tensorflowjs-1.7.4.post1-py3-none-any.whl (57 kB)
  Using cached tensorflow_cpu-2.2.0-cp37-cp37m-win_amd64.whl (189.3 MB)
